Dataset: https://www.kaggle.com/c/fake-news/data#

In [43]:
import pandas as pd

In [44]:
df=pd.read_csv("./fake-news/train.csv")

In [45]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [47]:
df.isna().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [48]:
df.dropna(inplace=True)

In [49]:
X=df.drop('label',axis=1)
y=df['label']

In [121]:
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,LSTM,Dense,Dropout

In [51]:
voc_size=5000

In [52]:
messages=X.copy()

In [53]:
messages.reset_index(inplace=True)
messages.drop('index',inplace=True,axis=1)
messages

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
18280,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
18281,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
18282,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
18283,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [83]:
import nltk
from nltk.corpus import stopwords
import re
import string

In [58]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

In [91]:
corpus=[]
for i in range(len(messages)):
    review=re.sub('[^A-Za-z]',' ',messages['title'][i])
    review=[ps.stem(word.lower()) for word in nltk.word_tokenize(review) if word.lower() not in stopwords.words("english")]
    review=' '.join(review)
    corpus.append(review)
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [98]:
onehot=[one_hot(word,voc_size) for word in corpus]
onehot

[[1849, 86, 2947, 1461, 2963, 1320, 1244, 100, 1694, 1555],
 [3673, 1771, 2900, 4027, 3804, 4091, 3289],
 [2715, 2011, 1382, 3301],
 [1877, 2549, 1486, 2295, 4209, 2021],
 [3447, 3804, 3567, 1812, 3824, 2090, 3804, 2513, 1754, 2900],
 [1734,
  758,
  2977,
  4236,
  4859,
  977,
  4258,
  3619,
  3781,
  2660,
  1408,
  4925,
  2842,
  977,
  3289],
 [2861, 220, 81, 2285, 3661, 1749, 3497, 2680, 2123, 529, 4898],
 [2698, 3226, 1155, 3636, 1257, 3238, 977, 1731, 2123, 529, 4898],
 [4246, 661, 3774, 4028, 1855, 1224, 4059, 1753, 977, 470],
 [3993, 2696, 2701, 1869, 2498, 937, 4069, 1988],
 [2941, 2796, 1299, 2091, 234, 1325, 3882, 2917, 4980, 2341, 2161],
 [2295, 1850, 2963, 1224, 977, 1257],
 [658, 4156, 3022, 913, 1169, 4886, 1259, 2674, 3221],
 [107, 1981, 4003, 3044, 4275, 1681, 566, 2123, 529, 4898],
 [3439, 3718, 1946, 2462, 54, 2123, 529, 4898],
 [75, 902, 1572, 3774, 4456, 2663, 1273, 529, 18, 4372],
 [226, 2474, 1771],
 [3559, 3150, 3906, 3160, 977, 2235, 261, 3289],
 [1940, 912

In [101]:
embedded_docs=pad_sequences(onehot,20,padding='pre')
# print(embedded_docs)

In [103]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 1849,
         86, 2947, 1461, 2963, 1320, 1244,  100, 1694, 1555], dtype=int32)

In [122]:
model=Sequential()
model.add(Embedding(5000,40,input_length=20))     #40->no of features
model.add(Dropout(0.3))
model.add(LSTM(100))            #one layer with 100 neurons
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
dropout (Dropout)            (None, 20, 40)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [123]:
import numpy as np
X=np.array(embedded_docs)
y=np.array(y)

In [124]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [127]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=20,batch_size=64,verbose=1)

Epoch 1/20
192/192 [==============================] - 7s 36ms/step - loss: 0.0761 - accuracy: 0.9736 - val_loss: 0.3013 - val_accuracy: 0.9112
Epoch 2/20
192/192 [==============================] - 8s 42ms/step - loss: 0.0524 - accuracy: 0.9825 - val_loss: 0.4039 - val_accuracy: 0.9115
Epoch 3/20
192/192 [==============================] - 8s 43ms/step - loss: 0.0358 - accuracy: 0.9875 - val_loss: 0.4085 - val_accuracy: 0.9044
Epoch 4/20
192/192 [==============================] - 8s 41ms/step - loss: 0.0279 - accuracy: 0.9912 - val_loss: 0.4128 - val_accuracy: 0.9092
Epoch 5/20
192/192 [==============================] - 8s 43ms/step - loss: 0.0219 - accuracy: 0.9938 - val_loss: 0.5622 - val_accuracy: 0.9092
Epoch 6/20
192/192 [==============================] - 9s 49ms/step - loss: 0.0224 - accuracy: 0.9929 - val_loss: 0.5196 - val_accuracy: 0.9084
Epoch 7/20
192/192 [==============================] - 9s 46ms/step - loss: 0.0146 - accuracy: 0.9951 - val_loss: 0.5243 - val_accuracy: 0.9107

In [128]:
from sklearn.metrics import confusion_matrix,classification_report
pred=model.predict(X_test)
pred = np.round(pred).astype(int)
pred
print(confusion_matrix(y_test,pred))
print(classification_report(y_test,pred))

[[3077  342]
 [ 209 2407]]
              precision    recall  f1-score   support

           0       0.94      0.90      0.92      3419
           1       0.88      0.92      0.90      2616

    accuracy                           0.91      6035
   macro avg       0.91      0.91      0.91      6035
weighted avg       0.91      0.91      0.91      6035

